In [1]:
print('OK!')

OK!


In [2]:
!pip install pypdf

  Using cached pypdf-4.2.0-py3-none-any.whl.metadata (7.4 kB)
Using cached pypdf-4.2.0-py3-none-any.whl (290 kB)


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os

/Users/nilaypatel/anaconda3/envs/mchatbot/lib/python3.8/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
api_key = os.getenv('PINECONE_API_KEY')
api_env = os.getenv('PINECONE_API_ENV')

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf('/Users/nilaypatel/End-to-End-Medical-Chatbot-Using-Llama2/data')

In [12]:
print(len(extracted_data))

637


In [6]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [41]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [13]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

In [15]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [44]:
def generate_embeddings():
    vector_embeddings = []
    for i in range(len(text_chunks)):
        vector_embeddings.append(embeddings.embed_query(text_chunks[i].page_content))
    return vector_embeddings


In [16]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [52]:
from pinecone import Pinecone
pc = Pinecone(api_key = api_key)


In [53]:
index = pc.Index('mchatbot')
v = generate_embeddings()
vectors = []
ids = []
for i in range(len(v)):
    ids.append('vec'+str(i+1))
for i in range(len(v)):
    vectors.append({'id':ids[i],'values':v[i]})



In [57]:
print(v[0])

[0.0017460581148043275, -0.03350289911031723, -0.03290392458438873, 0.007168038282543421, -0.01460327859967947, 0.010261868126690388, -0.011515314690768719, 0.22930210828781128, -0.023232391104102135, 0.004120429512113333, -0.036560941487550735, 0.08592110127210617, 0.012972128577530384, 0.05221787095069885, -0.1023261770606041, -0.0031389270443469286, -0.012686857022345066, 0.00047185903531499207, -0.028485851362347603, -0.05025915428996086, 0.01155101228505373, 0.07780641317367554, 0.09282822906970978, -0.013797281309962273, -0.016935022547841072, -0.025955891236662865, -0.0495651438832283, -0.04613136872649193, 0.0072905379347503185, -0.01355320867151022, 0.03843952342867851, 0.06280476599931717, 0.018353847786784172, 0.008242879062891006, 0.001715547521598637, -0.03986182063817978, -0.011638627387583256, 0.016446225345134735, 0.025595564395189285, 0.09104611724615097, 0.029672672972083092, -0.05416030064225197, -0.045765653252601624, -0.013853880576789379, 0.025773564353585243, 0.0

In [58]:
batch_size = 100
for i in range(len(vectors)):
    vctrs = vectors[i:i+batch_size]
    index.upsert(vctrs)

In [51]:
!pip install pinecone-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [59]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [60]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [62]:
lm=CTransformers(model="/Users/nilaypatel/End-to-End-Medical-Chatbot-Using-Llama2/model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [64]:
qa=RetrievalQA.from_chain_type(
    llm=lm, 
    chain_type="stuff", 
    retriever=.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

NameError: name 'docsearch' is not defined

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])